In [48]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2

In [49]:
# Path to the folders containing the sound files
folders = ['./Sound Source/Angry', './Sound Source/Calm', './Sound Source/Happy', './Sound Source/Sad']


In [50]:
# Path to save the spectrograms
save_path = './My Spectrograms'

In [51]:
# Create Train and Test folders
os.makedirs(os.path.join(save_path, 'Train'), exist_ok=True)
os.makedirs(os.path.join(save_path, 'Test'), exist_ok=True)

In [52]:
import librosa

def preprocess_audio(audio_path, sr=22050, duration=2, n_mels=128, image_size=(600, 400)):
    # Load audio file
    audio, _ = librosa.load(audio_path, sr=sr, duration=duration, mono=True)

    # Convert audio to spectrogram
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

    # Resize spectrogram to match the input shape of the model
    spectrogram = cv2.resize(spectrogram, dsize=image_size[::-1], interpolation=cv2.INTER_CUBIC)


    # Convert single-channel spectrogram to three-channel image
    spectrogram_rgb = np.stack((spectrogram,) * 3, axis=-1)




    return spectrogram_rgb[:, :, :3]

In [53]:
def normalize_spectrogram(spectrogram):
    # Normalize the spectrogram data
    normalized_spectrogram = ((spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram)) * 255).astype(np.uint8)
    return normalized_spectrogram

In [54]:
# Iterate over each folder
for folder in folders:
    # Get a list of all sound files in the folder
    files = [f for f in os.listdir(folder) if f.endswith('.wav')]
    
    # Split the files into Train and Test sets
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    # Process Train files
    for file in train_files:
        # Load the audio file
        audio_path = os.path.join(folder, file)
        spectrogram = preprocess_audio(audio_path)
        
        # Normalize the spectrogram
        spectrogram = normalize_spectrogram(spectrogram)  # Function to normalize the spectrogram

        # Save the spectrogram
        save_folder = os.path.join(save_path, 'Train', os.path.basename(folder))
        os.makedirs(save_folder, exist_ok=True)
        cv2.imwrite(os.path.join(save_folder, f'{file[:-4]}.png'), spectrogram)
        
    # Process Test files
    for file in test_files:
        # Load the audio file
        audio_path = os.path.join(folder, file)
        spectrogram = preprocess_audio(audio_path)

        # Normalize the spectrogram
        spectrogram = normalize_spectrogram(spectrogram)  # Function to normalize the spectrogram
        
        # Save the spectrogram
        save_folder = os.path.join(save_path, 'Test', os.path.basename(folder))
        os.makedirs(save_folder, exist_ok=True)
        cv2.imwrite(os.path.join(save_folder, f'{file[:-4]}.png'), spectrogram)